# Homolog BinaryMap Encoding

## Import `Python` modules

In [1]:
import pandas as pd
import re
import json
import binarymap as bmap
import matplotlib.pyplot as plt
import seaborn as sns
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax.experimental import sparse
from jaxopt import ProximalGradient
import jaxopt
import numpy as onp
from scipy.stats import pearsonr

from collections import defaultdict
from timeit import default_timer as timer

%matplotlib inline
import multidms

## Strategy for converting mutations to be relative to the reference homolog

In [9]:
# reference = "MT"
homologs = {
    "1" : "M-G",
    "2" : "MPP"
}

In [15]:
func_score_dict = {
    'condition' : ["1","1","1","1", "2","2","2","2","2","2"],
    'aa_substitutions' : ['M1E', 'G3R', 'G3P', 'M1W', 'M1E', 'P3R', 'P3G', 'M1E P3G', 'M1E P3R', 'P2T'],
    'func_score' : [2, -7, -0.5, 2.3, 1, -5, 0.4, 2.7, -2.7, 0.3],
}
func_score_df = pd.DataFrame(func_score_dict)
func_score_df

,condition,aa_substitutions,func_score
0,1,M1E,2.0
1,1,G3R,-7.0
2,1,G3P,-0.5
3,1,M1W,2.3
4,2,M1E,1.0
5,2,P3R,-5.0
6,2,P3G,0.4
7,2,M1E P3G,2.7
8,2,M1E P3R,-2.7
9,2,P2T,0.3


In [11]:
from multidms.model import global_epistasis

In [12]:
global_epistasis

frozendict.frozendict({'predict': Partial(<CompiledFunction of <function abstract_epistasis at 0x7f58c19ed3a0>>, φ=<CompiledFunction of <function φ_shift at 0x7f58c19f2b80>>, g=<CompiledFunction of <function sigmoidal_global_epistasis at 0x7f58c19f2f70>>, t=<CompiledFunction of <function identity_activation at 0x7f58c19f23a0>>), 'objective': Partial(<CompiledFunction of <function gamma_corrected_cost_smooth at 0x7f58c19edb80>>, f=Partial(<CompiledFunction of <function abstract_epistasis at 0x7f58c19ed3a0>>, φ=<CompiledFunction of <function φ_shift at 0x7f58c19f2b80>>, g=<CompiledFunction of <function sigmoidal_global_epistasis at 0x7f58c19f2f70>>, t=<CompiledFunction of <function identity_activation at 0x7f58c19f23a0>>)), 'proximal': <CompiledFunction of <function lasso_lock_prox at 0x7f58c19ed790>>})

In [14]:
mdms = multidms.Multidms(
    func_score_df,
    multidms.model.global_epistasis["predict"],
    multidms.model.global_epistasis["objective"],
    multidms.model.global_epistasis["proximal"],
    alphabet = multidms.AAS_WITHSTOP,
    reference = "1"
)

Found 1 site(s) lacking data in at least one condition.
1 of the 10 variants were removed because they had mutations at the above sites, leaving 9 variants.


100%|███████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 250.95it/s]


There were 0 cache hits in total for condition 2.


ValueError: Non-hashable static arguments are not supported. An error occurred during a call to 'abstract_epistasis' while trying to hash an object of type <class 'dict'>, {'β': DeviceArray([ 0.08086788, -0.38624702, -0.37565558,  1.66897423], dtype=float64), 'C_ref': DeviceArray([5.], dtype=float64), 'S': DeviceArray([0., 0., 0., 0.], dtype=float64), 'C': DeviceArray([0.], dtype=float64), 'γ': DeviceArray([0.], dtype=float64), 'α': {'ge_scale': DeviceArray([10.], dtype=float64), 'ge_bias': DeviceArray([-10.], dtype=float64)}}. The error was:
TypeError: unhashable type: 'dict'
